In [55]:
import pandas as pd
from bs4 import BeautifulSoup

In [56]:
df = pd.read_csv("hackernews.csv", parse_dates=["timestamp"])
df

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,NaN,NaN,The worst case for kickstarter is that they fa...,NaN,AJ007,NaN,1347981310,2012-09-18 15:15:10+00:00,comment,4538506,4537746.0,NaN,NaN,NaN
1,NaN,NaN,I&#x27;d have to say the same for Facebook. Mo...,NaN,jorgecurio,NaN,1456001084,2016-02-20 20:44:44+00:00,comment,11141867,11141158.0,NaN,NaN,NaN
2,NaN,NaN,&#62; You can't get San Francisco running effi...,NaN,Cushman,NaN,1359100263,2013-01-25 07:51:03+00:00,comment,5114503,5114256.0,NaN,NaN,NaN
3,NaN,NaN,&gt;illegal to transmit encrypted data over th...,NaN,aoeuasdf,NaN,1633726031,2021-10-08 20:47:11+00:00,comment,28803994,28803811.0,NaN,NaN,NaN
4,NaN,NaN,I actually think ARkit will be big precisely b...,NaN,erikpukinskis,NaN,1501892529,2017-08-05 00:22:09+00:00,comment,14933475,14932842.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4802,NaN,NaN,Or split into steam-the-platform-the-company a...,NaN,jrkatz,NaN,1564069670,2019-07-25 15:47:50+00:00,comment,20526386,20525938.0,NaN,NaN,NaN
4803,NaN,NaN,This isn&#x27;t as serious of a proposal as so...,NaN,yajoe,NaN,1387069124,2013-12-15 00:58:44+00:00,comment,6907828,6907262.0,NaN,NaN,NaN
4804,NaN,NaN,Ex coworkers. Last time I got a job tradition...,NaN,VLM,NaN,1378900686,2013-09-11 11:58:06+00:00,comment,6366593,6365783.0,NaN,NaN,NaN
4805,NaN,NaN,I predict that 0.10.x will be with us for a lo...,NaN,erichocean,NaN,1389925140,2014-01-17 02:19:00+00:00,comment,7073895,7073044.0,NaN,NaN,NaN


In [57]:
def extract_prediction(text):
    soup = BeautifulSoup(text)
    text = soup.text
    text = " ".join(text.split())
    pred_idx = text.find("I predict")
    dot_idx = text[pred_idx:].find(".")
    
    return text[pred_idx:pred_idx + dot_idx + 1]

In [58]:
df["prediction"] = df["text"].apply(extract_prediction)
df["year"] = df["timestamp"].dt.year
df = df[df["prediction"] != ""]

In [59]:
df.describe()

,score,time,id,parent,descendants,ranking,deleted,year
count,26.000000,4.680000e+03,4.680000e+03,4.654000e+03,24.000000,0.0,0.0,4680.000000
mean,4.115385,1.485184e+09,1.540349e+07,1.542911e+07,5.166667,NaN,NaN,2016.586752
std,3.614714,1.235184e+08,9.508852e+06,9.503724e+06,7.069018,NaN,NaN,3.923744
min,1.000000,1.175584e+09,8.462000e+03,8.407000e+03,-1.000000,NaN,NaN,2007.000000
25%,1.250000,1.388594e+09,6.995582e+06,7.015414e+06,0.750000,NaN,NaN,2014.000000
50%,2.500000,1.504021e+09,1.512503e+07,1.515935e+07,3.000000,NaN,NaN,2017.000000
75%,6.250000,1.590776e+09,2.335245e+07,2.336161e+07,6.500000,NaN,NaN,2020.000000
max,13.000000,1.658561e+09,3.220121e+07,3.220104e+07,31.000000,NaN,NaN,2022.000000


In [60]:
rnd = df.sample(1)
print(f"\"{rnd['prediction'].values[0]}\" -{rnd['by'].values[0]} ({rnd['year'].values[0]})")

"I predict we won't see much good come out of this initiative." -unalone (2009)


In [61]:
import json

generate = True

if generate:
    for i, d in enumerate(df[["prediction", "by", "year"]].to_dict(orient="records")):
        with open(f"../predictions/{i}", "w") as f:
            try:
                json.dump(d, f, ensure_ascii=False, indent=4)
            except UnicodeEncodeError as e:
                print(f"{i}\t{e}")


2617	'charmap' codec can't encode character '\ufff8' in position 90: character maps to <undefined>
